In [1]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split



In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv("/content/drive/MyDrive/augmented dataset/task1__eng_augmented.csv")
X_text = df['text'].tolist()
y_labels = df['polarization'].tolist()

In [14]:
print(df)

                                               id  \
0            eng_973938b90b0ff5d87d35a582f83f5c89   
1            eng_07dfd4600426caca6e2c5883fcbea9ea   
2            eng_f14519ff2302b6cd47712073f13bc461   
3            eng_e48b7e7542faafa544ac57b64bc80daf   
4            eng_7c581fb77bce8033aeba3d6dbd6273eb   
...                                           ...   
3343  eng_d118cc13746b8b3aa3066a621ccc13f9_aug667   
3344  eng_db607ce17c6337a53b97573b01100a12_aug668   
3345  eng_af2299a6d1a26c7bda07835fda65be6d_aug669   
3346  eng_67b71772937b16c9480050a767a28980_aug670   
3347  eng_d97075660229fab54952acf046dc7260_aug671   

                                                   text  polarization  
0              is defending imperialism in the dnd chat             0  
1     Still playing with this. I am now following Ra...             0  
2     .senate.gov Theres 3 groups out there Republic...             0  
3     "ABC MD, David Anderson, said the additional f...             0  
4  

In [6]:
model_name = "Qwen/Qwen3-Embedding-0.6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
embedding_model = AutoModel.from_pretrained(model_name).to(device)
embedding_model.eval()

for p in embedding_model.parameters():
    p.requires_grad = False

def mean_pooling(model_output, attention_mask):
    token_embeds = model_output.last_hidden_state
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeds.size()).float()
    sum_embeddings = torch.sum(token_embeds * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

In [7]:
def get_all_embeddings(texts, model, tokenizer, device, batch_size=32):
    all_embs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch_texts = texts[i:i+batch_size]
        enc = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            outputs = model(**enc)
            embeddings = mean_pooling(outputs, enc['attention_mask'])
        all_embs.append(embeddings.cpu())
    return torch.cat(all_embs, dim=0)


X_embeddings = get_all_embeddings(X_text, embedding_model, tokenizer, device)
y_tensor = torch.tensor(y_labels, dtype=torch.long)



Embedding: 100%|██████████| 105/105 [00:46<00:00,  2.27it/s]


In [8]:
class Classifier(nn.Module):
    def __init__(self, embed_dim, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.model(x)


In [9]:
# k = 5
# skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

# X = X_embeddings.numpy()
# y = y_tensor.numpy()
# num_classes = len(np.unique(y))

# all_acc = []
# all_macro_f1 = []

# for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
#     print(f"\n=== Fold {fold+1} ===")

#     # Split
#     X_train, X_test = X[train_idx], X[test_idx]
#     y_train, y_test = y[train_idx], y[test_idx]

#     # Tensors
#     X_train_t = torch.tensor(X_train, dtype=torch.float32)
#     y_train_t = torch.tensor(y_train, dtype=torch.long)
#     X_test_t = torch.tensor(X_test, dtype=torch.float32)
#     y_test_t = torch.tensor(y_test, dtype=torch.long)

#     # DataLoader
#     train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=32, shuffle=True)

#     # Initialize classifier
#     classifier = Classifier(X_train_t.shape[1], num_classes).to(device)
#     optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
#     criterion = nn.CrossEntropyLoss()

#     # Train
#     classifier.train()
#     for epoch in range(5):
#         for bx, by in train_loader:
#             bx, by = bx.to(device), by.to(device)
#             optimizer.zero_grad()
#             logits = classifier(bx)
#             loss = criterion(logits, by)
#             loss.backward()
#             optimizer.step()

#     # Evaluate
#     classifier.eval()
#     with torch.no_grad():
#         logits = classifier(X_test_t.to(device))
#         y_pred = torch.argmax(logits, dim=1).cpu().numpy()

#     acc = np.mean(y_pred == y_test)
#     macro_f1 = f1_score(y_test, y_pred, average='macro')

#     all_acc.append(acc)
#     all_macro_f1.append(macro_f1)

#     print(f"Fold {fold+1} - Accuracy: {acc:.4f} - Macro F1: {macro_f1:.4f}")

# # Cross-validation summary
# print(f"\n=== CV Summary ===")
# print(f"Average Accuracy: {np.mean(all_acc):.4f} ± {np.std(all_acc):.4f}")
# print(f"Average Macro F1: {np.mean(all_macro_f1):.4f} ± {np.std(all_macro_f1):.4f}")

In [19]:
X = X_embeddings.cpu().numpy()      # ensure numpy
y = y_tensor.cpu().numpy()

num_classes = len(np.unique(y))
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print("Train:", len(X_train), "Val:", len(X_val), "Test:", len(X_test))

# =========================================
# 3. Convert to tensors
# =========================================
X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)

X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_t = torch.tensor(y_val, dtype=torch.long).to(device)

X_test_t = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_t = torch.tensor(y_test, dtype=torch.long).to(device)

train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=32, shuffle=True)

# =========================================
# 4. Initialize classifier
# =========================================
classifier = Classifier(X_train_t.shape[1], num_classes).to(device)
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

# =========================================
# 5. Train classifier
# =========================================
classifier.train()
for epoch in range(30):
    for bx, by in train_loader:
        optimizer.zero_grad()
        logits = classifier(bx)
        loss = criterion(logits, by)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1} : {loss}")


Train: 2343 Val: 502 Test: 503
Epoch 1 : 0.5520896315574646
Epoch 2 : 0.8727965950965881
Epoch 3 : 0.3829401135444641
Epoch 4 : 0.6367412209510803
Epoch 5 : 0.4852500259876251
Epoch 6 : 0.011349841021001339
Epoch 7 : 0.46310940384864807
Epoch 8 : 0.046836137771606445
Epoch 9 : 0.15236493945121765
Epoch 10 : 0.3259749114513397
Epoch 11 : 0.12528716027736664
Epoch 12 : 0.3551623225212097
Epoch 13 : 0.24076102674007416
Epoch 14 : 0.07058129459619522
Epoch 15 : 0.402755469083786
Epoch 16 : 0.17089073359966278
Epoch 17 : 0.07294666767120361
Epoch 18 : 0.06241479516029358
Epoch 19 : 0.016786616295576096
Epoch 20 : 0.046426527202129364
Epoch 21 : 0.02233854867517948
Epoch 22 : 0.023388249799609184
Epoch 23 : 0.058644942939281464
Epoch 24 : 0.011534571647644043
Epoch 25 : 0.007949207909405231
Epoch 26 : 0.01911274529993534
Epoch 27 : 0.00047637769603170455
Epoch 28 : 0.010841690003871918
Epoch 29 : 0.14815391600131989
Epoch 30 : 0.014960057102143764


In [22]:
from sklearn.metrics import classification_report

classifier.eval()
with torch.no_grad():
    test_logits = classifier(X_test_t)
    test_preds = torch.argmax(test_logits, dim=1).cpu().numpy()

print("\n=== Classification Report ===")
print(classification_report(y_test, test_preds, digits=4))



=== Classification Report ===
              precision    recall  f1-score   support

           0     0.8208    0.7817    0.8008       252
           1     0.7909    0.8287    0.8093       251

    accuracy                         0.8052       503
   macro avg     0.8059    0.8052    0.8051       503
weighted avg     0.8059    0.8052    0.8051       503



In [12]:
# Suppose your classifier is trained and on device
model_path = "classifier_split.pth"
torch.save(classifier.state_dict(), model_path)
print(f"Classifier saved to {model_path}")


Classifier saved to classifier_split.pth
